In [2]:
import pandas as pd
import pickle
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression, LinearRegression

from collections import namedtuple
import numpy as np
from sklearn_pandas import DataFrameMapper

from sklearn.externals import joblib


# Display more rows and get rid of the margins
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Display mulitiple values from each cell
# https://stackoverflow.com/a/42476224
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Start rpy2 session
import patsy
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri

rpy2.robjects.numpy2ri.activate()

In [4]:
# Install the needed rlibraries, SLOW
robjects.r('''
source("https://bioconductor.org/biocLite.R")
biocLite("sva", ask = FALSE, suppressUpdates = FALSE)
''')
robjects.r('''
source("https://bioconductor.org/biocLite.R")
biocLite("BiocParallel", ask = FALSE, suppressUpdates = FALSE)
''')

/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: trying URL 'https://bioconductor.org/packages/3.4/bioc/src/contrib/BiocInstaller_1.24.0.tar.gz'

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Content type 'application/x-gzip'
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning:  length 17756 bytes (17 KB)

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: =
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: 

  res = super(Function, self).

/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning:  length 3763400 bytes (3.6 MB)

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: downloaded 3.6 MB


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: trying URL 'https://cran.rstudio.com/src/contrib/BH_1.66.0-1.tar.gz'

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning:  length 11583445 bytes (11.0 MB)

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: downloaded 11.0 MB


  res = super(Fun

/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: downloaded 1.8 MB


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: trying URL 'https://bioconductor.org/packages/3.4/bioc/src/contrib/Biobase_2.34.0.tar.gz'

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning:  length 1655200 bytes (1.6 MB)

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: downloaded 1.6 MB


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: trying URL 'https://bioconducto

array(['sva'],
      dtype='<U3')

/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Installing package(s) ‘BiocParallel’

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: also installing the dependencies ‘lambda.r’, ‘futile.options’, ‘futile.logger’, ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: trying URL 'https://cran.rstudio.com/src/contrib/lambda.r_1.2.tar.gz'

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning:  length 26315 bytes (25 KB)

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robject

array(['BiocParallel'],
      dtype='<U12')

In [5]:
# Load the combat function

robjects.r('library(sva)')
robjects.r('library(BiocParallel)')
robjects.r('''modifiedComBat <- function (dat, batch, mod = NULL, par.prior = FALSE, prior.plots = FALSE, 
           mean.only = FALSE, ref.batch = NULL, BPPARAM = bpparam("SerialParam"), dat2 = NULL, batch2 = NULL, mod2 = NULL) 
{
  if (mean.only == TRUE) {
    message("Using the 'mean only' version of ComBat")
  }
  if (length(dim(batch)) > 1) {
    stop("This version of ComBat only allows one batch variable")
  }
  if (is.null(batch2)) {
    batch <- as.factor(batch)
  }
  else {
    n1 <- length(batch)
    allbatch <- as.factor(c(batch, batch2))
    batch <- allbatch[1:n1]
    batch2 <- allbatch[(n1 + 1):length(allbatch)]
  }
  batchmod <- model.matrix(~-1 + batch)
  if (!is.null(dat2)) batchmod2 <- model.matrix(~-1 + batch2)
  if (!is.null(ref.batch)) {
    if (!(ref.batch %in% levels(batch))) {
      stop("reference level ref.batch is not one of the levels of the batch variable")
    }
    cat("Using batch =", ref.batch, "as a reference batch (this batch won't change)\n")
    ref <- which(levels(as.factor(batch)) == ref.batch)
    batchmod[, ref] <- 1
    if (!is.null(dat2)) {
      ref2 <- which(levels(as.factor(batch2)) == ref.batch)
      batchmod2[, ref2] <- 1
    }
  }
  else {
    ref <- NULL
  }
  message("Found", nlevels(batch), "batches")
  n.batch <- nlevels(batch)
  batches <- list()
  for (i in 1:n.batch) {
    batches[[i]] <- which(batch == levels(batch)[i])
  }
  if (!is.null(dat2)) {
    batches2 <- list()
    for (i in 1:n.batch) {
      batches2[[i]] <- which(batch2 == levels(batch)[i])
    }
  }
  n.batches <- sapply(batches, length)
  if (any(n.batches == 1)) {
    mean.only = TRUE
    message("Note: one batch has only one sample, setting mean.only=TRUE")
  }
  n.array <- sum(n.batches)
  design <- cbind(batchmod, mod)
  check <- apply(design, 2, function(x) all(x == 1))
  if (!is.null(ref)) {
    check[ref] <- FALSE
  }
  design <- as.matrix(design[, !check])
  if (!is.null(dat2)) {
    n.batches2 <- sapply(batches2, length)
    n.array2 <- sum(n.batches2)
    design2 <- cbind(batchmod2, mod2)
    check2 <- apply(design2, 2, function(x) all(x == 1))
    if (!is.null(ref)) {
      check2[ref] <- FALSE
    } 
    design2 <- as.matrix(design2[, !check2])
  }
  message("Adjusting for", ncol(design) - ncol(batchmod), "covariate(s) or covariate level(s)")
  if (qr(design)$rank < ncol(design)) {
    if (ncol(design) == (n.batch + 1)) {
      stop("The covariate is confounded with batch! Remove the covariate and rerun ComBat")
    }
    if (ncol(design) > (n.batch + 1)) {
      if ((qr(design[, -c(1:n.batch)])$rank < ncol(design[, 
                                                          -c(1:n.batch)]))) {
        stop("The covariates are confounded! Please remove one or more of the covariates so the design is not confounded")
      }
      else {
        stop("At least one covariate is confounded with batch! Please remove confounded covariates and rerun ComBat")
      }
    }
  }
  NAs <- any(is.na(dat))
  if (NAs) {
    message(c("Found", sum(is.na(dat)), "Missing Data Values"), 
            sep = " ")
  }
  cat("Standardizing Data across genes\n")
  
  if (!NAs) {
    B.hat <- solve(crossprod(design), tcrossprod(t(design), 
                                                 as.matrix(dat)))
  }
    else {
      B.hat <- apply(dat, 1, Beta.NA, design)
    }
  if (!is.null(ref.batch)) {
    grand.mean <- t(B.hat[ref, ])
  }
    else {
      grand.mean <- crossprod(n.batches/n.array, B.hat[1:n.batch, 
                                                       ])
    }
  if (!NAs) {
    if (!is.null(ref.batch)) {
      ref.dat <- dat[, batches[[ref]]]
      var.pooled <- ((ref.dat - t(design[batches[[ref]], 
                                         ] %*% B.hat))^2) %*% rep(1/n.batches[ref], n.batches[ref])
    }
    else {
      var.pooled <- ((dat - t(design %*% B.hat))^2) %*% 
        rep(1/n.array, n.array)
    }
  }
    else {
      if (!is.null(ref.batch)) {
        ref.dat <- dat[, batches[[ref]]]
        var.pooled <- rowVars(ref.dat - t(design[batches[[ref]], 
                                                 ] %*% B.hat), na.rm = TRUE)
      }
      else {
        var.pooled <- rowVars(dat - t(design %*% B.hat), 
                              na.rm = TRUE)
      }
    }
  stand.mean <- t(grand.mean) %*% t(rep(1, n.array))
  if (!is.null(dat2)) stand.mean2 <- t(grand.mean) %*% t(rep(1, n.array2))
  if (!is.null(design)) {
    tmp <- design
    tmp[, c(1:n.batch)] <- 0
    stand.mean <- stand.mean + t(tmp %*% B.hat)
    if (!is.null(dat2)) {
      tmp2 <- design2
      tmp2[, c(1:n.batch)] <- 0
      stand.mean2 <- stand.mean2 + t(tmp2 %*% B.hat)
    }
  }
  s.data <- (dat - stand.mean)/(sqrt(var.pooled) %*% t(rep(1, 
                                                           n.array)))
  if (!is.null(dat2)) {
    s.data2 <- (dat2 - stand.mean2)/(sqrt(var.pooled) %*% t(rep(1, 
                                                             n.array2)))
  }
  message("Fitting L/S model and finding priors")
  batch.design <- design[, 1:n.batch]
  if (!is.null(dat2)) batch.design2 <- design2[, 1:n.batch]
  if (!NAs) {
    gamma.hat <- solve(crossprod(batch.design), tcrossprod(t(batch.design), 
                                                           as.matrix(s.data)))
  }
    else {
      gamma.hat <- apply(s.data, 1, Beta.NA, batch.design)
    }
  delta.hat <- NULL
  for (i in batches) {
    if (mean.only == TRUE) {
      delta.hat <- rbind(delta.hat, rep(1, nrow(s.data)))
    }
    else {
      delta.hat <- rbind(delta.hat, rowVars(s.data[, i], 
                                            na.rm = TRUE))
    }
  }
  gamma.bar <- rowMeans(gamma.hat)
  t2 <- rowVars(gamma.hat)
  a.prior <- apply(delta.hat, 1, sva:::aprior)
  b.prior <- apply(delta.hat, 1, sva:::bprior)
  if (prior.plots && par.prior) {
    par(mfrow = c(2, 2))
    tmp <- density(gamma.hat[1, ])
    plot(tmp, type = "l", main = expression(paste("Density Plot of First Batch ", 
                                                  hat(gamma))))
    xx <- seq(min(tmp$x), max(tmp$x), length = 100)
    lines(xx, dnorm(xx, gamma.bar[1], sqrt(t2[1])), col = 2)
    qqnorm(gamma.hat[1, ], main = expression(paste("Normal Q-Q Plot of First Batch ", 
                                                   hat(gamma))))
    qqline(gamma.hat[1, ], col = 2)
    tmp <- density(delta.hat[1, ])
    xx <- seq(min(tmp$x), max(tmp$x), length = 100)
    tmp1 <- list(x = xx, y = sva:::dinvgamma(xx, a.prior[1], b.prior[1]))
    plot(tmp, typ = "l", ylim = c(0, max(tmp$y, tmp1$y)), 
         main = expression(paste("Density Plot of First Batch ", 
                                 hat(delta))))
    lines(tmp1, col = 2)
    invgam <- 1/qgamma(1 - ppoints(ncol(delta.hat)), a.prior[1], 
                       b.prior[1])
    qqplot(invgam, delta.hat[1, ], main = expression(paste("Inverse Gamma Q-Q Plot of First Batch ", 
                                                           hat(delta))), ylab = "Sample Quantiles", xlab = "Theoretical Quantiles")
    lines(c(0, max(invgam)), c(0, max(invgam)), col = 2)
  }
  gamma.star <- delta.star <- matrix(NA, nrow = n.batch, ncol = nrow(s.data))
  if (par.prior) {
    message("Finding parametric adjustments")
    results <- BiocParallel:::bplapply(1:n.batch, function(i) {
      if (mean.only) {
        gamma.star <- postmean(gamma.hat[i, ], gamma.bar[i], 
                               1, 1, t2[i])
        delta.star <- rep(1, nrow(s.data))
      }
      else {
        temp <- sva:::it.sol(s.data[, batches[[i]]], gamma.hat[i, 
                                                               ], delta.hat[i, ], gamma.bar[i], t2[i], a.prior[i], 
                             b.prior[i])
        gamma.star <- temp[1, ]
        delta.star <- temp[2, ]
      }
      list(gamma.star = gamma.star, delta.star = delta.star)
    }, BPPARAM = BPPARAM)
    for (i in 1:n.batch) {
      gamma.star[i, ] <- results[[i]]$gamma.star
      delta.star[i, ] <- results[[i]]$delta.star
    }
  }
    else {
      message("Finding nonparametric adjustments")
      results <- BiocParallel:::bplapply(1:n.batch, function(i) {
        if (mean.only) {
          delta.hat[i, ] = 1
        }
        temp <- sva:::int.eprior(as.matrix(s.data[, batches[[i]]]), 
                           gamma.hat[i, ], delta.hat[i, ])
        list(gamma.star = temp[1, ], delta.star = temp[2, 
                                                       ])
      }, BPPARAM = BPPARAM)
      for (i in 1:n.batch) {
        gamma.star[i, ] <- results[[i]]$gamma.star
        delta.star[i, ] <- results[[i]]$delta.star
      }
    }
  if (!is.null(ref.batch)) {
    gamma.star[ref, ] <- 0
    delta.star[ref, ] <- 1
  }
  message("Adjusting the Data\n")
  bayesdata <- s.data
  j <- 1
  for (i in batches) {
    bayesdata[, i] <- (bayesdata[, i] - t(batch.design[i, 
                                                       ] %*% gamma.star))/(sqrt(delta.star[j, ]) %*% t(rep(1, 
                                                                                                           n.batches[j])))
    j <- j + 1
  }
  bayesdata <- (bayesdata * (sqrt(var.pooled) %*% t(rep(1, 
                                                        n.array)))) + stand.mean
  if (!is.null(ref.batch)) {
    bayesdata[, batches[[ref]]] <- dat[, batches[[ref]]]
  }
  if (!is.null(dat2)) {
    bayesdata2 <- s.data2
    j <- 1
    for (i in batches2) {
      bayesdata2[, i] <- (bayesdata2[, i] - t(batch.design2[i, 
                                                         ] %*% gamma.star))/(sqrt(delta.star[j, ]) %*% t(rep(1, 
                                                                                                             n.batches2[j])))
      j <- j + 1
    }
    bayesdata2 <- (bayesdata2 * (sqrt(var.pooled) %*% t(rep(1, 
                                                          n.array2)))) + stand.mean2
    if (!is.null(ref.batch)) {
      bayesdata2[, batches2[[ref]]] <- dat2[, batches2[[ref]]]
    }
    return(list(corrected = bayesdata, alpha = grand.mean, beta.hat = B.hat, gamma.star = gamma.star, delta.star = delta.star,
                corrected2 = bayesdata2))
  }
  return(list(corrected = bayesdata, alpha = grand.mean, beta.hat = B.hat, gamma.star = gamma.star, delta.star = delta.star))
}
''')
combat = robjects.r('modifiedComBat')

/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: mgcv

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: nlme

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: This is mgcv 1.8-23. For overview type 'help("mgcv-package")'.

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: genefilter

  res = super(Function, self).__call__(*new_args, **new_kwargs)


array(['sva', 'genefilter', 'mgcv', 'nlme', 'BiocInstaller', 'tools',
       'stats', 'graphics', 'grDevices', 'utils', 'datasets', 'methods',
       'base'],
      dtype='<U13')

array(['BiocParallel', 'sva', 'genefilter', 'mgcv', 'nlme',
       'BiocInstaller', 'tools', 'stats', 'graphics', 'grDevices', 'utils',
       'datasets', 'methods', 'base'],
      dtype='<U13')

<SignatureTranslatedFunction - Python:0x2b3ea7bb6208 / R:0x2b3eb83cbfe0>

# Simulate some data

In [31]:
def run_combat(feats, meta, model="~1",
              feats_test=None, meta_test=None, model_test=None, par_prior=False):
    model_matrix = patsy.dmatrix(model, meta)
    fmat = np.array(feats).T
    rbatch = robjects.IntVector(pd.Categorical(meta.site).codes)
    
    if (meta_test is not None) and (feats_test is not None):
        if model_test is None:
            model_test = model
        model_matrix_test = patsy.dmatrix(model_test, meta_test)
        fmat_test = np.array(feats_test).T
        rbatch_test = robjects.IntVector(pd.Categorical(meta_test.site).codes)
        combat_result = combat(dat=fmat, batch=rbatch, mod=model_matrix,
                               dat2=fmat_test, batch2=rbatch_test, mod2=model_matrix_test, par_prior=par_prior)
    else:
        combat_result = combat(dat = fmat, batch = rbatch, mod = model_matrix)
    combat_result = [np.array(cr) for cr in combat_result]
    return combat_result

In [20]:
n_samples = 800
n_cols = 100
n_class = 2
train_test_ratio = 1

In [30]:
#Generate Random Data, you'll need to do this outside the parallelization, 
# so maybe create random arrays that have 100 arrays on the last dimension and iterate through those
simY_r = np.random.randint(0,n_class, n_samples)
simX_r = np.random.normal(size = (n_samples,n_cols))
simY_e = np.random.randint(0, n_class, n_samples//train_test_ratio)
simX_e = np.random.normal(size = (n_samples//train_test_ratio, n_cols))

#Because of that randomization bug, you'll need to start your run_sim function here
# def run_sim(simX_r, simY_r, simY_e, simY_e)

# Make the columns strings so the dataframe mapper doesn't choke
simX_r = pd.DataFrame(data=simX_r, columns=np.arange(n_cols).astype(str))
metric_cols = simX_r.columns
simX_r['site'] = simY_r.astype(str)
simX_e = pd.DataFrame(data=simX_e, columns=metric_cols)
simX_e['site'] = simY_e.astype(str)


In [32]:
cb_meta_cols = ['site']
simX = pd.concat((simX_r, simX_e)).reset_index(drop=True)
simX_r_cb1 = simX_r.copy(deep=True)
simX_e_cb1 = simX_e.copy(deep=True)
simX_r_cb2 = simX_r.copy(deep=True)
simX_e_cb2 = simX_e.copy(deep=True)
simX_r_cb3 = simX_r.copy(deep=True)
simX_e_cb3 = simX_e.copy(deep=True)

# combat application 1: whole dataset
combat_res = run_combat(simX.loc[:, metric_cols], simX.loc[:, ['site']],
                        model='~1')

simX_r_cb2.loc[:, metric_cols] = combat_res[0].T[:len(simX_r)]
simX_e_cb2.loc[:, metric_cols] = combat_res[0].T[len(simX_r):]

# combat application 2: train learn test apply
combat_res = run_combat(simX_r.loc[:, metric_cols], simX_r.loc[:, ['site']],
                        feats_test= simX_e.loc[:, metric_cols],
                        meta_test= simX_e.loc[:, ['site']],
                        model='~1')

simX_r_cb2.loc[:, metric_cols] = combat_res[0].T
simX_e_cb2.loc[:, metric_cols] = combat_res[5].T

# combat application 3: train learn test apply
combat_res_r = run_combat(simX_r.loc[:, metric_cols], simX_r.loc[:, ['site']],
                        model='~1')
combat_res_e = run_combat(simX_e.loc[:, metric_cols], simX_e.loc[:, ['site']],
                        model='~1')

simX_r_cb3.loc[:, metric_cols] = combat_res_r[0].T
simX_e_cb3.loc[:, metric_cols] = combat_res_e[0].T

/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Found2batches

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Adjusting for0covariate(s) or covariate level(s)

  res = super(Function, self).__call__(*new_args, **new_kwargs)


Standardizing Data across genes



/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Fitting L/S model and finding priors

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Finding nonparametric adjustments

  res = super(Function, self).__call__(*new_args, **new_kwargs)


Standardizing Data across genes

Standardizing Data across genes

Standardizing Data across genes



/data/nielsond/python/envs/abcd_mine/lib/python3.6/site-packages/rpy2/robjects/functions.py:106: UserWarning: Adjusting the Data


  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [33]:
mapper = DataFrameMapper([([str(nv)],preprocessing.StandardScaler()) for nv in metric_cols])
sim_clf = Pipeline([('preprocessing', mapper),
                    ('clf', LogisticRegression(multi_class='multinomial', solver='saga', max_iter = 100000,
                                               penalty='l2',
                                               C=1,
                                               fit_intercept=True))])

sim_clf.fit(simX_r, simY_r)
sim_clf.score(simX_r, simY_r)
sim_clf.score(simX_e, simY_e)

sim_clf.fit(simX_r_cb1, simY_r)
sim_clf.score(simX_r_cb1, simY_r)
sim_clf.score(simX_e_cb1, simY_e)

sim_clf.fit(simX_r_cb2, simY_r)
sim_clf.score(simX_r_cb2, simY_r)
sim_clf.score(simX_e_cb2, simY_e)

sim_clf.fit(simX_r_cb3, simY_r)
sim_clf.score(simX_r_cb3, simY_r)
sim_clf.score(simX_e_cb3, simY_e)

Pipeline(memory=None,
     steps=[('preprocessing', DataFrameMapper(default=False, df_out=False,
        features=[(['0'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['1'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['2'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['3'], Sta...penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False))])

0.63500000000000001

0.47249999999999998

Pipeline(memory=None,
     steps=[('preprocessing', DataFrameMapper(default=False, df_out=False,
        features=[(['0'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['1'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['2'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['3'], Sta...penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False))])

0.63500000000000001

...

0.58374999999999999

0.42249999999999999

Pipeline(memory=None,
     steps=[('preprocessing', DataFrameMapper(default=False, df_out=False,
        features=[(['0'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['1'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['2'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['3'], Sta...penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False))])

0.58374999999999999

0.46875

# TODO:
We need to parallelize this to run ~ 100 times each for train_test_ratio of 1, train_test_ratio of 5, n_class= 2, n_class = 10
I'd keep njobs at 30 or lower on felix

You can use something like this run signature
just need a funciton for running the simulation that outputs the two scores for the raw data and the three methods of combat correction
cb_var_res = joblib.Parallel(n_jobs=n_jobs)(joblib.delayed(function)(*pp) for pp in parameters)